In [38]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from scipy.spatial.distance import pdist, cdist
import operator as op

In [47]:
def geo_distance(c_i, c_j):
    lng1, lat1 = c_i[0], c_i[1]
    lng2, lat2 = c_j[0], c_j[1]
    #lng1,lat1,lng2,lat2 = (120.12802999999997,30.28708,115.86572000000001,28.7427)
    lng1, lat1, lng2, lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=round(distance,3)
    # print(c_i, c_j, distance)
    return distance

In [48]:
def count_points(t):
    traj = t[0].split(' ')
    pad_cnt = traj.count('[PAD]')
    cnt = len(traj) - pad_cnt
    return cnt

In [56]:
def rangeQuery(t, c, c_number):
    # 补一个token->(lon,lat)的逻辑
    cnt = 0
    # coo_list = []
    dist_list = []
    dist_list_1 = []
    traj = t[0].split(' ')
    # print('traj:', traj)
    for i in range(len(traj)):
        if traj[i] == '[PAD]':
            continue
        token = int(traj[i])
        # print(token)
        # coo = tuple((loc_index.iloc[token][0], loc_index.iloc[token][1]))
        distance_1 = dm.values[token][c_number]
        # distance = geo_distance(coo, (c[0], c[1]))
        if distance_1 < c[2]:
            cnt += 1
        # coo_list.append((loc_index.iloc[token][0], loc_index.iloc[token][1]))
        # dist_list.append(distance)
        dist_list_1.append(distance_1)

    # for coo in coo_list:
    #     distance = geo_distance(coo, (c[0], c[1]))
    #     dist_list_2.append(distance)
    #     # if distance < c[2]:
    #     #     cnt += 1

    # 用于检查两种distance计算方法的结果一不一样
    # for i in range(len(dist_list_1)):
    #     print(dist_list[i], dist_list_1[i])
    return cnt

In [42]:
loc_index = pd.read_csv('./loc_index.csv',index_col=0)
cluster_center = pd.read_csv('../data/cluster_center.csv',header=None)
cluster_center.columns = ['lon','lat']

In [43]:
all_traj = pd.read_hdf('./all_traj.h5', key='data')

In [58]:
all_traj

,trajectory,user_index,year,month,day
0,22042 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3
2,15374 15429 15665 15466 15874 15502 15653 1554...,0,2009,4,4
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6
...,...,...,...,...,...
1726,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13
1727,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15
1728,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2
1729,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12


In [60]:
sgm = 0.8
lbd = 0.6

In [49]:
radius = pdist(cluster_center.values.tolist(), lambda c_i, c_j: geo_distance(c_i, c_j)).min()
cluster_center['radius'] = radius * sgm

In [50]:
loc_index

,lon,lat
0,115.416875,40.444661
1,115.418200,40.445192
2,115.481076,39.981526
3,115.495228,40.021426
4,115.496418,40.030745
...,...,...
68584,117.508193,41.003427
68585,117.508193,40.443502
68586,117.508228,39.727523
68587,117.508230,39.945472


In [51]:
%%time
locs = loc_index.values.tolist()
centers = cluster_center[['lon','lat']].values.tolist()
dm = cdist(locs, centers, lambda loc, center: geo_distance(loc, center))

CPU times: user 1.95 s, sys: 0 ns, total: 1.95 s
Wall time: 1.94 s


In [52]:
# 每个点（loc_index）到每个质心的距离
dm = pd.DataFrame(dm, columns=[('C_'+str(i))for i in range(len(cluster_center))])
dm

In [54]:
dm.index.name = 'loc_index'
dm.to_hdf('./distance_matrix.h5', key='data')

In [65]:
%%time
dm = pd.read_hdf('./distance_matrix.h5', key='data')
label = len(all_traj)*[-1]
fallen_table = []

# go through every traj
for j, t in enumerate(all_traj.values):
    cnt_list = []
    fallen_rate_list = []
    points_num = count_points(t)
    # calculate fallenRate to each cluster
    for i, c in enumerate(cluster_center.values):
        fallenPoints = rangeQuery(t, c, i)
        fallenRate = fallenPoints/points_num
        fallen_rate_list.append((fallenRate,i))
        cnt_list.append(fallenPoints)
        # break
    cnt_list.append(points_num)
    fallen_table.append(cnt_list)
    fallen_rate_list.sort(key=lambda x: x[0], reverse=True)

    # choose cluster with max fallenRate
    # label[j] = fallen_rate_list[0][1]

    # use lambda to determine cluster
    label[j] = fallen_rate_list[0][1] if fallen_rate_list[0][0]>lbd else -1
    print("Traj.No:", j, 'Max fallenRate: %.2f'%fallen_rate_list[0][0], 'Label:', label[j])
    print(cnt_list)
    print(fallen_rate_list)
    print()

    # break # for check single traj

Traj.No: 0 Max fallenRate: 0.38 Label: -1
[5, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 13]
[(0.38461538461538464, 0), (0.38461538461538464, 5), (0.0, 1), (0.0, 2), (0.0, 3), (0.0, 4), (0.0, 6), (0.0, 7), (0.0, 8), (0.0, 9), (0.0, 10), (0.0, 11)]

Traj.No: 1 Max fallenRate: 0.65 Label: 0
[15, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 23]
[(0.6521739130434783, 0), (0.13043478260869565, 5), (0.043478260869565216, 3), (0.0, 1), (0.0, 2), (0.0, 4), (0.0, 6), (0.0, 7), (0.0, 8), (0.0, 9), (0.0, 10), (0.0, 11)]

Traj.No: 2 Max fallenRate: 0.66 Label: 0
[27, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 41]
[(0.6585365853658537, 0), (0.024390243902439025, 3), (0.0, 1), (0.0, 2), (0.0, 4), (0.0, 5), (0.0, 6), (0.0, 7), (0.0, 8), (0.0, 9), (0.0, 10), (0.0, 11)]

Traj.No: 3 Max fallenRate: 0.22 Label: -1
[4, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 18]
[(0.2222222222222222, 0), (0.1111111111111111, 5), (0.05555555555555555, 3), (0.0, 1), (0.0, 2), (0.0, 4), (0.0, 6), (0.0, 7), (0.0, 8), (0.0, 9), (0.0, 10), (0.0, 11)]

Traj.No: 4 Max

In [66]:
# the points number that each center contains for each trajectory
fallen_table

[[5, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 13],
 [15, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 23],
 [27, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 41],
 [4, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 18],
 [4, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 14],
 [2, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 12],
 [3, 0, 0, 1, 0, 5, 0, 0, 0, 0, 0, 0, 16],
 [2, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 15],
 [9, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 13],
 [2, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 15],
 [0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 17],
 [8, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 12],
 [6, 0, 0, 1, 0, 8, 0, 0, 0, 0, 0, 0, 20],
 [4, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 16],
 [6, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 13],
 [3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 13],
 [6, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 12],
 [9, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 12],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 18],
 [6, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 14],
 [7, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 12],
 [8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 13],
 [12, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 21],
 [1, 0,

In [110]:
df_fallen_table = pd.DataFrame(fallen_table, columns=[('C_'+str(i))for i in range(len(cluster_center))]+['points_num'])
df_fallen_table['sum'] = df_fallen_table.iloc[:,:-1].sum(axis=1)
df_fallen_table['points_no_label'] = df_fallen_table['points_num'] - df_fallen_table['sum']
df_fallen_table.index.name = 'traj_no'
df_fallen_table.to_hdf('./fallen_table.h5', key='data')

In [70]:
df_fallen_table = pd.read_hdf('./fallen_table.h5', key='data')
df_fallen_table
# points_sum: 轨迹有多少个points
# sum: 轨迹中有多少个点落在这12个cluster内
# sum == 0 则表示整条轨迹没有任何一个点可以被分到这12类里

,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,C_11,points_num,sum,points_no_label
traj_no,,,,,,,,,,,,,,,
0,5,0,0,0,0,5,0,0,0,0,0,0,13,10,3
1,15,0,0,1,0,3,0,0,0,0,0,0,23,19,4
2,27,0,0,1,0,0,0,0,0,0,0,0,41,28,13
3,4,0,0,1,0,2,0,0,0,0,0,0,18,7,11
4,4,0,0,2,0,3,0,0,0,0,0,0,14,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,2,0,0,0,0,2,0,0,0,0,0,0,12,4,8
1727,1,0,0,0,0,6,0,0,0,0,0,1,18,8,10
1728,1,0,0,0,0,9,0,0,0,0,0,0,14,10,4


In [71]:
print(df_fallen_table['sum'].value_counts())

0     367
1     112
3     110
2     102
6      97
4      89
8      80
7      71
9      71
5      70
10     67
12     64
14     61
13     58
11     56
15     42
16     36
17     29
18     27
20     18
19     17
21     17
23     13
24      9
22      8
25      7
29      6
28      5
26      4
32      4
27      3
30      3
34      2
48      2
38      1
31      1
35      1
40      1
Name: sum, dtype: int64


In [74]:
# 给all_traj加label
all_traj['label'] = label
all_traj ['']

,trajectory,user_index,year,month,day,label
0,22042 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31,-1
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3,0
2,15374 15429 15665 15466 15874 15502 15653 1554...,0,2009,4,4,0
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5,-1
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6,-1
...,...,...,...,...,...,...
1726,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13,-1
1727,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15,-1
1728,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2,5
1729,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12,-1


In [82]:
all_traj.to_hdf('all_traj_labeled_σ_{sgm}_λ_{lbd}.h5'.format(sgm=sgm, lbd=lbd), key='data')

In [83]:
all_traj

,trajectory,user_index,year,month,day,label
0,22042 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31,-1
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3,0
2,15374 15429 15665 15466 15874 15502 15653 1554...,0,2009,4,4,0
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5,-1
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6,-1
...,...,...,...,...,...,...
1726,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13,-1
1727,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15,-1
1728,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2,5
1729,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12,-1


In [81]:
import pandas as pd
all_traj_labeled = pd.read_hdf("all_traj_labeled_σ_{sgm}_λ_{lbd}.h5".format(sgm=sgm, lbd=lbd), key='data')

FileNotFoundError: File all_traj_labeled_σ_0.8_λ_0.6.h5 does not exist

In [80]:
all_traj_labeled

,trajectory,user_index,year,month,day,label
0,22042 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31,-1
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3,0
2,15374 15429 15665 15466 15874 15502 15653 1554...,0,2009,4,4,0
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5,-1
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6,-1
...,...,...,...,...,...,...
1726,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13,-1
1727,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15,-1
1728,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2,5
1729,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12,-1


In [78]:
cluster_value_counts = pd.DataFrame(columns=['Num','Percentage'])
cluster_value_counts['Num'] = all_traj_labeled\
          ['label'].value_counts().sort_index()

In [79]:
cluster_value_counts

,Num,Percentage
-1,1456,NaN
0,52,NaN
1,7,NaN
2,9,NaN
3,87,NaN
4,3,NaN
5,46,NaN
6,9,NaN
7,13,NaN
8,21,NaN


In [21]:
cluster_value_counts['Percentage'] =cluster_value_counts['Num']/len(all_traj_labeled)
cluster_value_counts['Percentage'] = cluster_value_counts['Percentage'].apply(lambda x: format(x, '.2%'))
cluster_value_counts.index = ['Cluster']

TypeError: Index(...) must be called with a collection of some kind, 'Cluster' was passed

In [29]:
cluster_value_counts.index.name = 'Cluster'
cluster_value_counts.to_csv('./cluster_value_counts.csv')